In [1]:
import tvm
from tvm import relay
from tvm import autotvm
import tvm.contrib.graph_runtime as runtime
from tvm.contrib import util
from tvm.contrib.util import tempdir
import numpy as np

In [2]:
target = tvm.target.arm_cpu("rasp3b")
target_host = 'llvm -device=arm_cpu -target=arm-linux-gnueabihf -mattr=+neon'

#target = 'llvm'

In [3]:
x = relay.var('x', shape=[1, 4096], dtype='int16')
#w = relay.var('w', shape=[32, 32, 3, 3], dtype='int16')
#w_np = np.random.normal(size=[32, 32, 3, 3]).astype('float32').astype('int16')
w = relay.var('w', shape=[4096, 4096], dtype='int16')
#w = relay.var('w', dtype='int16')
w_np = np.random.normal(size=[4096, 4096]).astype('int16')
x_np = np.random.normal(size=[1, 4096]).astype('int16')

#y = relay.nn.conv2d(x, w, data_layout='NHWC', kernel_size=[3, 3], kernel_layout='OIHW')
bpw = relay.nn.bitpack(w, bit_axis=1, pack_axis=1, bits=1, pack_type='uint8')
#bpw = relay.nn.bitpack(w, bits=1, pack_axis=2, bit_axis=4, pack_type='uint8')
y = relay.nn.bitserial_dense(x, bpw, units=4096, data_bits=1, weight_bits=1, unipolar=False, pack_dtype='uint8')
y_func = relay.Function([x, w], y)

In [4]:
print(y_func.body)

v0.0.3
free_var %x: Tensor[(1, 4096), int16]
free_var %w: Tensor[(4096, 4096), int16]
%0 = nn.bitpack(%w, bit_axis=1, pack_type="int8");
nn.bitserial_dense(%x, %0, units=4096, pack_dtype="int8", out_dtype="int16", unipolar=False)


In [5]:
params = {'w': w_np}
with relay.build_config(opt_level=0):
    graph, lib, params = relay.build_module.build(y_func, target=target, params=params)

Cannot find config for target=llvm -device=arm_cpu -model=bcm2837 -target=armv7l-linux-gnueabihf -mattr=+neon, workload=('bitserial_dense', (1, 4096, 'int16'), (4096, 1, 512, 'int8'), 1, 1, 'int8', 'int16', 0). A fallback configuration is used, which may bring great performance regression.


TVMError: Traceback (most recent call last):
  [bt] (8) /home/tvm/build/libtvm.so(tvm::relay::backend::RelayBuildModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, std::shared_ptr<tvm::runtime::ModuleNode> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const+0x96a) [0x7f474b8d25fa]
  [bt] (7) /home/tvm/build/libtvm.so(tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::relay::Function, std::unordered_map<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, tvm::runtime::NDArray, std::hash<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > >, std::equal_to<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > >, std::allocator<std::pair<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const, tvm::runtime::NDArray> > > const&)+0x7c

In [23]:
tmp = util.tempdir()
lib_fname = tmp.relpath('net.tar')
lib.export_library(lib_fname)

In [24]:
remote = autotvm.measure.request_remote(
    'rpi3b', 'fleet.cs.washington.edu', 9190, timeout=10000)
#remote = tvm.rpc.LocalSession()

In [25]:
# upload the library to remote device and load it
remote.upload(lib_fname)
rlib = remote.load_module('net.tar')

# create the remote runtime module
ctx = remote.cpu(0)
module = runtime.create(graph, rlib, ctx)
# set parameter (upload params to the remote device. This may take a while)
module.set_input(**params)

In [26]:
module.set_input('x', x_np)
module.run()

In [27]:
 # Evaluate
print("Evaluate inference time cost...")
ftimer = module.module.time_evaluator("run", ctx, number=10, repeat=1)
prof_res = np.array(ftimer().results) * 1000  # Convert to milliseconds
print("Mean inference time (std dev): %.2f ms (%.2f ms)" %
      (np.mean(prof_res), np.std(prof_res)))

Evaluate inference time cost...
Mean inference time (std dev): 40.56 ms (0.00 ms)
